# TODO: Multi-Class Image Classification on Dogs Breed Dataset, with Amazon Sagemaker 

## I Fine Tuning a Pre-Trained Model on this dataset
## II Model Debugging and Profiling

This notebook lists all the steps that you need to complete the complete this project. You will need to complete all the TODOs in this notebook as well as in the README and the two python scripts included with the starter code.


**TODO**: Give a helpful introduction to what this notebook is for. Remember that comments, explanations and good documentation make your project informative and professional.

**Note:** This notebook has a bunch of code and markdown cells with TODOs that you have to complete. These are meant to be helpful guidelines for you to finish your project while meeting the requirements in the project rubrics. Feel free to change the order of these the TODO's and use more than one TODO code cell to do all your tasks.


In [42]:
# TODO: Install any packages that you might need
# For instance, you will need the smdebug package
!pip install smdebug

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 270 kB 29.1 MB/s            
     |████████████████████████████████| 83 kB 179 kB/s              
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


# Dataset

## We are using the Dogs Breed Multi-Class Classification Dataset

TODO: Explain what dataset you are using for this project. Maybe even give a small overview of the classes, class distributions etc that can help anyone not familiar with the dataset get a better understand of it.

## We have downloaded the dataset and uploaded it to AWS S3
- Uploaded Data to a S3 Bucket
- Checked all data is present currently
- Understood Distribution of Train-Validation-Test

### Dataset Link for Reference

https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip


In [2]:
# TODO: Import any packages that you might need
# For instance you will need Boto3 and Sagemaker
import sagemaker
import boto3

In [3]:
from sagemaker import get_execution_role
role = get_execution_role()

In [4]:
conn = boto3.client('s3')

### Checking All Data is uploaded correctly

In [38]:
bucket = 'sagemaker-studio-88wth0tideu'
subfolder = 'dogImages/train'
#subfolder = 'dogImages/valid'
#subfolder = 'dogImages/test'

In [40]:
contents = conn.list_objects(Bucket=bucket, Prefix=subfolder)['Contents']

In [41]:
uploaded_set = []
for f in contents:
    #print(f['Key'])
    uploaded_set.append(f['Key'])

In [7]:
original_set = []
f = open("dog_train.txt", "r+") #Checked for valid and test also
for line in f.readlines():
    #print(line)
    original_set.append(line[:-1])

In [8]:
original_set = set(original_set)

In [9]:
original_set = sorted(original_set)

In [82]:
uploaded_set = set(uploaded_set)

In [64]:
data_difference = original_set - uploaded_set

In [5]:
train_folders = []
f = open("dog_train_folders.txt", "r+")
for line in f.readlines():
    #print(line)
    train_folders.append(line[:-1])

In [19]:
train_folders = sorted(train_folders)

In [30]:
i = 0
files_folder_wise = []
for folder in train_folders:
    folder_files = []
    for f in original_set:
        if folder in f:
            folder_files.append(f)
            i+=1
    files_folder_wise.append(folder_files)
    
print(len(files_folder_wise))
print(i)

133
6680


In [43]:
f_cnt = 0
for folder in train_folders:
    #print(folder)
    sub_f = folder
    i+=1
    try:
        contents = conn.list_objects(Bucket=bucket, Prefix=sub_f)['Contents']
    except:
        print("Folder Doesn't exist")
        print(folder)
        f_cnt+=1
        continue
    uploaded_set = []
    for f in contents:
        #print(f['Key'])
        uploaded_set.append(f['Key'])
    uploaded_set = set(uploaded_set)
    original_set = set(files_folder_wise[f_cnt])
    diff = original_set - uploaded_set
    #print(diff) #For verification that all folders are complete
    f_cnt+=1

### Dataset Distribution Facts

#### Number of Classes (Breeds) = 133

In [55]:
len(files_folder_wise)

133

In [56]:
train_folders #Name of each Breed

['dogImages/train/001.Affenpinscher',
 'dogImages/train/002.Afghan_hound',
 'dogImages/train/003.Airedale_terrier',
 'dogImages/train/004.Akita',
 'dogImages/train/005.Alaskan_malamute',
 'dogImages/train/006.American_eskimo_dog',
 'dogImages/train/007.American_foxhound',
 'dogImages/train/008.American_staffordshire_terrier',
 'dogImages/train/009.American_water_spaniel',
 'dogImages/train/010.Anatolian_shepherd_dog',
 'dogImages/train/011.Australian_cattle_dog',
 'dogImages/train/012.Australian_shepherd',
 'dogImages/train/013.Australian_terrier',
 'dogImages/train/014.Basenji',
 'dogImages/train/015.Basset_hound',
 'dogImages/train/016.Beagle',
 'dogImages/train/017.Bearded_collie',
 'dogImages/train/018.Beauceron',
 'dogImages/train/019.Bedlington_terrier',
 'dogImages/train/020.Belgian_malinois',
 'dogImages/train/021.Belgian_sheepdog',
 'dogImages/train/022.Belgian_tervuren',
 'dogImages/train/023.Bernese_mountain_dog',
 'dogImages/train/024.Bichon_frise',
 'dogImages/train/025.Bl

In [50]:
train_set = []
f = open("dog_train.txt", "r+") 
for line in f.readlines():
    #print(line)
    train_set.append(line[:-1])
    
valid_set = []
f = open("dog_valid.txt", "r+") 
for line in f.readlines():
    #print(line)
    valid_set.append(line[:-1])
    
test_set = []
f = open("dog_test.txt", "r+") 
for line in f.readlines():
    #print(line)
    test_set.append(line[:-1])

#### Train Data = 6680 Images

In [51]:
len(train_set)

6680

#### Validation Data = 836 Images

In [52]:
len(valid_set)

835

#### Test Data = 836 Images

In [53]:
len(test_set)

836

#### Total Data = 8351 Images

In [54]:
len(train_set) + len(valid_set) + len(test_set)

8351

## Hyperparameter Tuning
**TODO:** This is the part where you will finetune a pretrained model with hyperparameter tuning. Remember that you have to tune a minimum of two hyperparameters. However you are encouraged to tune more. You are also encouraged to explain why you chose to tune those particular hyperparameters and the ranges.

**Note:** You will need to use the `hpo.py` script to perform hyperparameter tuning.

In [57]:
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)

In [58]:
from sagemaker.pytorch import PyTorch

In [60]:
#TODO: Declare your HP ranges, metrics etc.
hyperparameter_ranges = {
    "lr": ContinuousParameter(0.001, 0.1),
    "batch-size": CategoricalParameter([32, 64, 128, 256, 512]),
    "epochs": IntegerParameter(10,20)
}

In [67]:
#TODO: Create estimators for your HPs

estimator = PyTorch(
    entry_point="hpo.py",
    role=role,
    py_version='py36',
    framework_version="1.8",
    instance_count=1,
    instance_type="ml.m5.large"
)

objective_metric_name = "average test loss"
objective_type = "Minimize"
metric_definitions = [{"Name": "average test loss", "Regex": "Test set: Average loss: ([0-9\\.]+)"}]

tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=1,
    max_parallel_jobs=1,
    objective_type=objective_type,
)# TODO: Your HP tuner here

In [68]:
training_path = "s3://sagemaker-studio-88wth0tideu/dogImages/"

In [69]:
# TODO: Fit your HP Tuner
tuner.fit({"training": training_path}) 

............................................................................*


UnexpectedStatusException: Error for HyperParameterTuning job pytorch-training-220211-2038: Failed. Reason: All training jobs failed. Please take a look at the training jobs failures to get more details.

In [ ]:
# TODO: Get the best estimators and the best HPs

best_estimator = tuner.best_estimator()#TODO

#Get the hyperparameters of the best trained model
best_estimator.hyperparameters()

## Model Profiling and Debugging
TODO: Using the best hyperparameters, create and finetune a new model

**Note:** You will need to use the `train_model.py` script to perform model profiling and debugging.

In [ ]:
# TODO: Set up debugging and profiling rules and hooks

In [ ]:
# TODO: Create and fit an estimator

estimator = # TODO: Your estimator here

In [ ]:
# TODO: Plot a debugging output.

**TODO**: Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it?  
**TODO**: If not, suppose there was an error. What would that error look like and how would you have fixed it?

In [ ]:
# TODO: Display the profiler output

## Model Deploying

In [ ]:
# TODO: Deploy your model to an endpoint

predictor=estimator.deploy() # TODO: Add your deployment configuration like instance type and number of instances

In [ ]:
# TODO: Run an prediction on the endpoint

image = # TODO: Your code to load and preprocess image to send to endpoint for prediction
response = predictor.predict(image)

In [ ]:
# TODO: Remember to shutdown/delete your endpoint once your work is done
predictor.delete_endpoint()